In [1]:
#ALL IMPORTS
import numpy as np
import pandas as pd

In [2]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [9]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = {i for i in self.data.HomeTeam}
        #temp = {i for i in self.data.AwayTeam}
        #self.teams.add(temp)
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamsTotalGoals(self):
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG
            
        return teamTotalGoals
    
    def cleanData(self):
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"]
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def Test(self):
        print("Passed")

In [10]:
season1718 = Football(season19_20,"Hooaeg 19-20")
print(season1718.)

{'Bournemouth': 5, 'Liverpool': 0, 'Burnley': 5, 'Man United': 4, 'Newcastle': 6, 'Tottenham': 4, 'Chelsea': 3, 'Leicester': 2, 'Everton': 7, 'Crystal Palace': 6, 'Aston Villa': 7, 'Brighton': 6, 'West Ham': 6, 'Wolves': 2, 'Norwich': 9, 'Southampton': 8, 'Sheffield United': 3, 'Man City': 3, 'Arsenal': 3, 'Watford': 7}
